### 主程式
1. 設定資料庫 & 爬蟲設定
2. 顯示UI
3. 使用者操作
4. 讀取資料庫顯示在UI上

In [39]:
from sqlDB import sqlDB
from crawler3 import cwb
from datetime import date

from ipywidgets.widgets import Button, DatePicker, Dropdown, Label
from ipywidgets.widgets import Layout, HBox, VBox
from IPython.display import display, clear_output

# 設定下載日期範圍
start_date = date(2021,9,1)
end_date =date.today()
# 設定下載路徑
downloadFolder = r'C:\Downloads\cwb'
# 設定想要下載的測站
targetList = []
targetList.append('福山植物園')

# 創立物件 (cwb類別)
cwbInstance = cwb()
# 創建物件 (sqlDB類別)
dbInstance = sqlDB("cwb")


資料夾存在
Database Opened successfully
County, Table existed
Station, Table existed


In [48]:
align_kw = dict(
    _css = (('.widget-label', 'min-width', '20ex'),),
    margin = '0px 0px 5px 12px',
)

# 台灣城市
btn_initial = Button(
    description ='初始化',
    icon='bug',
    button_style='warning',
    layout=Layout(width='100px')
)

btn_download = Button(
    description ='下載',
    icon='download',
    button_style='warning',
    layout=Layout(width='100px')
)

drop_county = Dropdown(
    options=dbInstance.getTableList("County"),
    description='城市',
    disabled = False,
    layout=Layout(width='auto', align_items='center')
    
)

drop_station = Dropdown(
    options = dbInstance.getStationListatCounty("Station",0),
    description='測站',
    disabled = False,
    layout=Layout(width='auto', align_items='center')
)

def initial_click(b):
    cwbInstance.openBrowser()
    cwbInstance.getAllSelect()
    print('創建物件耶耶 上網瞜!!!')
    # 放置資料庫的Table (County)
    dbInstance.deleteTableAllRows("County")
    dbInstance.insertCountyTable(cwbInstance)
    # 放置資料庫的Table (Station)
    dbInstance.deleteTableAllRows("Station")
    dbInstance.insertStationTable(cwbInstance)

def download_click(b):
    #clear_output()
    print(drop_county.options[drop_county.value][0])
    print(drop_station.value)
    

def show_station(a):
    drop_station.options = dbInstance.getStationListatCounty("Station", drop_county.value)
    
drop_county.observe(show_station)
btn_initial.on_click(initial_click)
btn_download.on_click(download_click)

In [138]:
# https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_Flexible_Box_Layout/Aligning_Items_in_a_Flex_Container
box_layout_column = Layout(width='80%', display='flex', flex_flow='column', justify_content='space-around', align_items='flex-start')
box_layout_row = Layout(width='80%', display='flex', flex_flow='row', justify_content='space-between', align_items='center')
V_items1 = [drop_county, drop_station]
V_items2 = [btn_initial, btn_download]
H_items = [VBox(V_items1, layout = box_layout_column), VBox(V_items2, layout = box_layout_column)]

Box1 = HBox(H_items, layout = box_layout_row)

In [143]:
btn_table = Button(
    description ='表格',
    icon='table',
    button_style='warning',
    layout=Layout(width='100px')
)

btn_chart = Button(
    description ='曲線圖',
    icon='line-chart',
    button_style='warning',
    layout=Layout(width='100px')
)


dateset_picker_from = DatePicker(
        description='從',
        disabled=False,
    )

dateset_picker_to = DatePicker(
        description='到',
        disabled=False,
    )

box_layout_column = Layout(width='80%', display='flex', flex_flow='column', justify_content='space-around', align_items='flex-start')
box_layout_row = Layout(width='80%', display='flex', flex_flow='row', justify_content='space-between', align_items='center')
#box_layout_row2 = Layout(width='80%', display='flex', flex_flow='row', justify_content='space-between', align_items='center',border = "solid 1px")

V_items_show1 = [dateset_picker_from, dateset_picker_to]
V_items_show2 = [btn_table, btn_chart]
H_items_show1 = [VBox(V_items_show1, layout = box_layout_column), VBox(V_items_show2, layout = box_layout_column)]

Box2 = HBox(H_items_show1, layout = box_layout_row)

In [140]:
display(VBox([Box1,Box2]),layout = box_layout_column)

In [ ]:
# 創立物件 (cwb類別)
cwbInstance = cwb()
cwbInstance.openBrowser()
cwbInstance.getAllSelect()

In [ ]:
cwbInstance.downloadFolder = downloadFolder
cwbInstance.downloadList(targetList, start_date, end_date)

In [ ]:
cwbInstance.allStations

In [9]:
# 開啟下載的csv整理後存入db裏頭
import pandas as pd
filepath = "C:\Downloads\cwb\新北市_福山植物園 (Fushan Botanical Garden)_C0AH90\C0AH90-2021-09-05.csv"
df = pd.read_csv(filepath, parse_dates=True)
df

,觀測時間(hour),測站氣壓(hPa),海平面氣壓(hPa),氣溫(℃),露點溫度(℃),相對溼度(%),風速(m/s),風向(360degree),最大陣風(m/s),最大陣風風向(360degree),降水量(mm),降水時數(h),日照時數(h),全天空日射量(MJ/㎡),能見度(km),紫外線指數,總雲量(0~10)
0,ObsTime,StnPres,SeaPres,Temperature,Td dew point,RH,WS,WD,WSGust,WDGust,Precp,PrecpHour,SunShine,GloblRad,Visb,UVI,Cloud Amount
1,01,940.9,...,19.0,...,99,0.0,0,...,...,0.0,...,...,...,...,...,...
2,02,940.7,...,18.6,...,99,0.0,0,...,...,0.0,...,...,...,...,...,...
3,03,940.5,...,18.7,...,99,0.0,0,...,...,0.0,...,...,...,...,...,...
4,04,940.6,...,18.5,...,99,0.0,0,...,...,0.0,...,...,...,...,...,...
5,05,940.6,...,18.2,...,99,0.0,0,...,...,0.0,...,...,...,...,...,...
6,06,941.1,...,18.1,...,100,0.0,0,...,...,0.0,...,...,...,...,...,...
7,07,941.0,...,21.1,...,99,0.0,0,...,...,0.0,...,...,...,...,...,...
8,08,941.0,...,24.4,...,86,0.5,48,...,...,0.0,...,...,...,...,...,...
9,09,941.3,...,26.6,...,78,1.0,65,...,...,0.0,...,...,...,...,...,...
